# Imports and Mounting

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
import os
import random
import shutil
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision.models import ResNet18_Weights
import torch.optim as optim

from PIL import Image
from torch.utils.data import Dataset

!pip install tqdm
from tqdm import tqdm

!pip install rdkit
from rdkit import Chem
from rdkit.Chem.rdMolDescriptors import CalcMolFormula

from collections import defaultdict
import re


# Sampling the data

In [ ]:
project_root = "/content/drive/Shareddrives/CIS5190FinalProj"

handdrawn_root = "/content/drive/Shareddrives/CIS5190FinalProj/DECIMER_HDM_Dataset_Images"
computer_root = "/content/drive/Shareddrives/CIS5190FinalProj/Img2Mol"

In [ ]:
print(f"There are {len(os.listdir(handdrawn_root))} handdrawn images")
print(f"There are {len(os.listdir(computer_root))} computer generated images")

There are 5088 handdrawn images
There are 10880 computer generated images


In [ ]:
## Training : Validation : Testing = 7 : 2 : 1
NUMBER_OF_TRAINING_IMGS = 7000
NUMBER_OF_VAL_IMGS = NUMBER_OF_TRAINING_IMGS // 7 * 2
NUMBER_OF_TESTING_IMGS = NUMBER_OF_TRAINING_IMGS // 7

train_root = project_root + "/Train0.5"
val_root = project_root + "/Val0.5"
test_root = project_root + "/Test0.5"

## Creating image sets

In [ ]:
def create_new_directory(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory)

In [ ]:
## Randomly sample the images
def split_data(comp_gen_percentage=0.5):
    if comp_gen_percentage < 0.5:
        print("Computer generated percentage cannot be smaller than 0.5")
        return

    # Computer generated images
    num_comp_gen_imgs_train = int(comp_gen_percentage * NUMBER_OF_TRAINING_IMGS)
    num_comp_gen_imgs_val = int(comp_gen_percentage * NUMBER_OF_VAL_IMGS)
    num_comp_gen_imgs_test = int(comp_gen_percentage * NUMBER_OF_TESTING_IMGS)


    files = os.listdir(computer_root) # a list of names of computer-generated images
    random.shuffle(files)

    comp_gen_filenames_train = files[:num_comp_gen_imgs_train]
    comp_gen_filenames_val = files[num_comp_gen_imgs_train: num_comp_gen_imgs_train+num_comp_gen_imgs_val]
    comp_gen_filenames_test = files[num_comp_gen_imgs_train+num_comp_gen_imgs_val:]

    # Create the directories
    create_new_directory(train_root)
    create_new_directory(val_root)
    create_new_directory(test_root)

    # Copy the images into the directories
    for filename in comp_gen_filenames_train:
        shutil.copy(os.path.join(computer_root, filename), train_root)
    for filename in comp_gen_filenames_val:
        shutil.copy(os.path.join(computer_root, filename), val_root)
    for filename in comp_gen_filenames_test:
        shutil.copy(os.path.join(computer_root, filename), test_root)



    # Handdrawn images
    num_hand_imgs_train = int((1 - comp_gen_percentage) * NUMBER_OF_TRAINING_IMGS)
    num_hand_imgs_val = int((1 - comp_gen_percentage) * NUMBER_OF_VAL_IMGS)
    num_hand_imgs_test = int((1 - comp_gen_percentage) * NUMBER_OF_TESTING_IMGS)


    files = os.listdir(handdrawn_root)
    random.shuffle(files)

    hand_filenames_train = files[:num_hand_imgs_train]
    hand_filenames_val = files[num_hand_imgs_train: num_hand_imgs_train+num_hand_imgs_val]
    hand_filenames_test = files[num_hand_imgs_train+num_hand_imgs_val:]

    # Copy the images into the directories
    for filename in hand_filenames_train:
        shutil.copy(os.path.join(handdrawn_root, filename), train_root)
    for filename in hand_filenames_val:
        shutil.copy(os.path.join(handdrawn_root, filename), val_root)
    for filename in hand_filenames_test:
        shutil.copy(os.path.join(handdrawn_root, filename), test_root)

In [ ]:
## Only need to run this once
#split_data(1)

## Loading the data into a dataloader

### Get the labels

In [ ]:
def smiles_to_formula(smiles):
  mol = Chem.MolFromSmiles(smiles)
  formula = CalcMolFormula(mol)
  return formula

def formula_to_atoms(formula):
  atoms = defaultdict(int)
  elements = re.findall(r'([A-Z][a-z]*)(\d*)', formula)
  for element, count in elements:
    count = int(count) if count else 1
    atoms[element] += count
  return dict(atoms)


def smiles_to_atoms(smiles):
  formula = smiles_to_formula(smiles)
  return formula_to_atoms(formula)

def atoms_to_array(atoms):
  # List of all elements in the periodic table in order
  periodic_table = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne",
                    "Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K", "Ca",
                    "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn",
                    "Ga", "Ge", "As", "Se", "Br", "Kr", "Rb", "Sr", "Y", "Zr",
                    "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd", "In", "Sn",
                    "Sb", "Te", "I", "Xe", "Cs", "Ba", "La", "Ce", "Pr", "Nd",
                    "Pm", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb",
                    "Lu", "Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg",
                    "Tl", "Pb", "Bi", "Po", "At", "Rn", "Fr", "Ra", "Ac", "Th",
                    "Pa", "U", "Np", "Pu", "Am", "Cm", "Bk", "Cf", "Es", "Fm",
                    "Md", "No", "Lr", "Rf", "Db", "Sg", "Bh", "Hs", "Mt", "Ds",
                    "Rg", "Cn", "Nh", "Fl", "Mc", "Lv", "Ts", "Og"]
  # Initialize array with zeros for each element
  element_array = [0] * 118
  # Place the atom counts into the array based on the dictionary
  for element, count in atoms.items():
      if element in periodic_table:
          index = periodic_table.index(element)
          element_array[index] = count
  return element_array

In [ ]:
handdrawn_df = pd.read_csv(project_root + "/DECIMER_HDM_Dataset_SMILES.tsv", sep='\t')
handdrawn_smiles_dict = handdrawn_df.set_index('IDs')['SMILES'].to_dict()

# Process each SMILES to convert into atom arrays
handdrawn_atom_arrays = {key: atoms_to_array(smiles_to_atoms(value)) for key, value in handdrawn_smiles_dict.items()}

# To verify the transformation, print the first 5 elements of the transformed dictionary
for key in list(handdrawn_atom_arrays.keys())[:5]:
    print(key, ":", handdrawn_atom_arrays[key])


CDK_Depict_1_2 : [3, 0, 0, 0, 0, 6, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
CDK_Depict_1_4 : [6, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
CDK_Depict_1_5 : [45, 0, 0, 0, 0, 21, 0, 2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
# Test cases
def test_smiles_to_formula():
  print('here')
  print(smiles_to_formula("C"))
  print(smiles_to_formula("O"))
  print(smiles_to_formula("CCO"))
  print(smiles_to_formula("C(=O)C"))
  print(smiles_to_formula("[Na]"))
  print(smiles_to_formula("[Fe]"))
  print(smiles_to_formula("C(C(C(C)))"))
  print(smiles_to_formula("C(CC(CC))"))
  print(smiles_to_formula("C([C@@H](C(=O)O)N)S"))

# Run test cases
test_smiles_to_formula()

def test_formula_to_atoms():
  print('here')
  print(formula_to_atoms("C"))
  print(formula_to_atoms("O"))
  print(formula_to_atoms("CO2"))
  print(formula_to_atoms("H2O"))
  print(formula_to_atoms("C2H6O"))
  print(formula_to_atoms("C3H7NO2S"))

print("Other test:")
test_formula_to_atoms()

def test_smiles_to_atoms():
  print('here')
  print(smiles_to_atoms("C"))
  print(smiles_to_atoms("O"))
  print(smiles_to_atoms("CCO"))
  print(smiles_to_atoms("C(=O)C"))
  print(smiles_to_atoms("[Na]"))
  print(smiles_to_atoms("[Fe]"))
  print(smiles_to_atoms("C(C(C(C)))"))
  print(smiles_to_atoms("C(CC(CC))"))
  print(smiles_to_atoms("C([C@@H](C(=O)O)N)S"))

test_smiles_to_atoms()

here
CH4
H2O
C2H6O
C2H4O
Na
Fe
C4H10
C5H12
C3H7NO2S
Other test:
here
{'C': 1}
{'O': 1}
{'C': 1, 'O': 2}
{'H': 2, 'O': 1}
{'C': 2, 'H': 6, 'O': 1}
{'C': 3, 'H': 7, 'N': 1, 'O': 2, 'S': 1}
here
{'C': 1, 'H': 4}
{'H': 2, 'O': 1}
{'C': 2, 'H': 6, 'O': 1}
{'C': 2, 'H': 4, 'O': 1}
{'Na': 1}
{'Fe': 1}
{'C': 4, 'H': 10}
{'C': 5, 'H': 12}
{'C': 3, 'H': 7, 'N': 1, 'O': 2, 'S': 1}


In [ ]:
with open("/content/drive/Shareddrives/CIS5190FinalProj/Img2Mol_map.pkl", 'rb') as f:
    comp_gen_df = pickle.load(f)
comp_gen_smiles_dict = comp_gen_df.set_index('Image')['SMILES'].to_dict()
comp_gen_atom_arrays = {key: atoms_to_array(smiles_to_atoms(value)) for key, value in comp_gen_smiles_dict.items()}


In [ ]:
# Verify transformation for both datasets by printing the first 5 elements
print("Hand-drawn dataset first 5 atom arrays:")
for key in list(handdrawn_atom_arrays.keys())[:5]:
    print(key, ":", handdrawn_atom_arrays[key])

print("\nComputer-generated dataset first 5 atom arrays:")
for key in list(comp_gen_atom_arrays.keys())[:5]:
    print(key, ":", comp_gen_atom_arrays[key])

Hand-drawn dataset first 5 atom arrays:
CDK_Depict_1_2 : [3, 0, 0, 0, 0, 6, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
CDK_Depict_1_4 : [6, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
CDK_Depict_1_5 : [45, 0, 0, 0, 0, 21, 0, 2, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
## Combine the two dictionaries together
combined_mapping = handdrawn_atom_arrays.copy()

for key, value in comp_gen_atom_arrays.items():
    # Remove .png from the key
    key = key.replace('.png', '')
    combined_mapping[key] = value


In [ ]:
## Returns the label based on the filename of the image
def get_label(filename):
    return combined_mapping.get(filename, None)

In [ ]:
def validate_dataset(image_names, root_dir, label_dict):
    valid_image_names = []
    for img_name in image_names:
        img_path = os.path.join(root_dir, img_name)
        if os.path.isfile(img_path) and img_name.replace(".png", "") in label_dict:
            valid_image_names.append(img_name)
    return valid_image_names


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, label_dict, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_names = validate_dataset(os.listdir(root_dir), root_dir, label_dict)
        self.label_dict = label_dict

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        img_name = self.image_names[idx]
        img_path = os.path.join(self.root_dir, img_name)
        image = Image.open(img_path).convert('RGB')
        label = self.label_dict[img_name.replace(".png", "")]

        if self.transform:
            image = self.transform(image)

        label = torch.tensor(label, dtype=torch.float32)
        return image, label


In [ ]:
## Transformation
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:
train_dataset = CustomDataset(root_dir=train_root, label_dict=combined_mapping, transform=transform)
val_dataset = CustomDataset(root_dir=val_root, label_dict=combined_mapping, transform=transform)
test_dataset = CustomDataset(root_dir=test_root, label_dict=combined_mapping, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)


# Data Augmentation

In [ ]:
aug1 = transforms.Compose([
    transforms.RandomHorizontalFlip(),   # Random horizontal flip
    transforms.RandomRotation(degrees=15) # Random rotation by up to 15 degrees
])

aug2 = transforms.Compose([
    transforms.RandomResizedCrop(size=224),  # Random resized crop to 224x224
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jitter
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10)  # Random affine transformations
])

aug3 = transforms.Compose([
    transforms.RandomHorizontalFlip(),   # Random horizontal flip
    transforms.RandomRotation(degrees=15), # Random rotation by up to 15 degrees
    transforms.RandomResizedCrop(size=224),  # Random resized crop to 224x224
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Color jitter
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10)  # Random affine transformations
])

# Model Pipeline (Transfer Learning)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [ ]:
# Custom MSE Loss function
class CustomMSELoss(nn.Module):
    def __init__(self):
        super(CustomMSELoss, self).__init__()
        self.mse_loss = nn.MSELoss()

    def forward(self, output, target):
        loss = self.mse_loss(output, target)
        return loss

In [ ]:
def calculate_accuracy(output, target):
    output = torch.round(output).detach().cpu().numpy()
    target = target.detach().cpu().numpy()
    return np.mean(np.isclose(output, target, atol=1e-2))

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)

        model.eval()
        val_loss = 0.0
        val_accuracy = 0.0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_accuracy += calculate_accuracy(outputs, labels)

        val_loss = val_loss / len(val_loader)
        val_accuracy = val_accuracy / len(val_loader)

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')

In [ ]:
def test_model(model, test_loader, criterion):
    model.eval()
    test_loss = 0.0
    test_accuracy = 0.0

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(tqdm(test_loader, desc="Testing")):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            rounded_outputs = torch.round(outputs)
            test_accuracy += calculate_accuracy(rounded_outputs, labels)

            # Print some predictions and labels for debugging
            if i % 50 == 0:
                print(f'Batch {i}/{len(test_loader)}, Current Test Loss: {test_loss/(i+1):.4f}, Current Test Accuracy: {test_accuracy/(i+1):.4f}')
                print("Predictions:", rounded_outputs[:5].cpu().numpy())
                print("Targets:", labels[:5].cpu().numpy())

    test_loss = test_loss / len(test_loader)
    test_accuracy = test_accuracy / len(test_loader)

    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

## ResNet

In [ ]:
weights = ResNet18_Weights.DEFAULT
resnet = models.resnet18(weights=weights)
for param in resnet.parameters():
    param.requires_grad = False

num_features = resnet.fc.in_features
num_elements = 118
resnet.fc = nn.Linear(num_features, num_elements)

resnet = resnet.to(device)  # Move model to GPU

In [ ]:
resnet2 = models.resnet18(weights=weights)
for param in resnet2.parameters():
    param.requires_grad = False

num_features = resnet2.fc.in_features
num_elements = 118
resnet2.fc = nn.Linear(num_features, num_elements)

resnet2 = resnet2.to(device)  # Move model to GPU

In [ ]:
criterion = CustomMSELoss()
optimizer = torch.optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)

train_model(resnet, train_loader, val_loader, criterion, optimizer, num_epochs=5)

Epoch [1/5], Train Loss: 0.8399, Val Loss: 0.7762, Val Accuracy: 0.0261
Epoch [2/5], Train Loss: 0.7414, Val Loss: 0.7138, Val Accuracy: 0.0281
Epoch [3/5], Train Loss: 0.6892, Val Loss: 0.6733, Val Accuracy: 0.0290
Epoch [4/5], Train Loss: 0.6659, Val Loss: 0.6473, Val Accuracy: 0.0295
Epoch [5/5], Train Loss: 0.6357, Val Loss: 0.6318, Val Accuracy: 0.0312


In [ ]:
criterion = CustomMSELoss()
optimizer = torch.optim.SGD(resnet.parameters(), lr=0.001, momentum=0.9)

train_model(resnet2, train_loader, val_loader, criterion, optimizer, num_epochs=10)

KeyboardInterrupt: 

In [ ]:
test_model(resnet, test_loader, criterion)

Testing:   0%|          | 1/218 [00:00<01:01,  3.55it/s]

Batch 0/218, Current Test Loss: 0.3613, Current Test Accuracy: 0.9441
Predictions: [[19. -0. -0. -0.  0. 21.  3.  4.  0.  0.  0. -0. -0. -0.  0.  1. -0.  1.
  -0. -0. -0.  0. -0. -0.  0.  0. -0. -0.  0. -0.  0.  0. -1. -0. -0.  0.
   0.  0.  0. -0.  0. -0.  0.  0.  0.  0.  0. -0.  0.  0. -0. -0.  0. -0.
   1.  0. -0. -0. -1.  1.  1. -0.  0.  1.  0.  0. -0. -0. -0.  0.  0.  0.
  -0.  1.  0.  0. -0. -0. -0. -1. -0. -0. -0. -0.  0. -0. -0. -0. -0. -0.
  -0.  0.  0.  0.  0. -0. -0. -0. -0. -0.  0.  0. -0. -0.  0.  0. -0. -0.
  -0.  0.  0. -0.  0.  0.  0. -0.  0.  0.]
 [18.  0.  0. -0. -0. 19.  3.  4.  1.  0.  0.  0.  0. -0.  0.  1.  0. -0.
   0.  0.  0.  0.  0.  0. -0. -0.  1.  0. -0.  0.  0. -0. -0.  0. -0.  0.
   0.  0.  0.  0. -0.  0.  0. -0. -0. -0. -0. -0. -0. -0.  0.  0. -0.  0.
   0.  0. -0. -0. -1. -0.  0. -0.  0. -0.  0. -0.  0.  0. -0. -0.  0.  0.
   0.  0.  0. -0.  0. -0.  0. -0. -1.  0.  0. -0. -0.  0. -0.  0. -0. -0.
  -0.  0.  0. -1. -0. -0.  0.  0. -0. -0. -0. -0. -0.  0.  0

Testing:  13%|█▎        | 28/218 [00:09<01:03,  2.99it/s]


KeyboardInterrupt: 